In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_raw = pd.read_csv('order_brush_order.csv')
df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
df_raw.event_time = pd.to_datetime(df_raw.event_time)

In [ ]:
test_df = df_raw[df_raw.shopid == df_raw.shopid[0]]
test_df

In [ ]:
group = test_df[(test_df.event_time >= test_df.event_time[0]) & (test_df.event_time <= (test_df.event_time[0] + pd.Timedelta('1H')))]
group

In [ ]:
group.orderid.count() / group.userid.nunique()

In [ ]:
def loop_event(df_shop):

    def each_start_time(start_time):
        end_time = start_time + pd.Timedelta('1H')
        group = df_shop[(df_shop.event_time >= start_time) & (df_shop.event_time <= end_time)]
        score = group.orderid.count() / group.userid.nunique()

        if score >= 3:
            # order brushing
            counts = group.groupby('userid').count().orderid
            meow = counts.to_dict()
            meow = [{'userid': key, 'count': value} for key, value in meow.items()]
            return meow

    hello = df_shop.event_time.apply(each_start_time)
    result = hello[hello.astype(bool)]
    result = result.sum()

    if result == 0:
        return 0

    result = pd.DataFrame(result)
    result = result.groupby('userid').sum()['count']

    max_occur = result.max()
    result = result[result==max_occur].index.tolist()
    result.sort()
    result = [str(x) for x in result]

    return '&'.join(result)

loop_event(df_raw[df_raw.shopid==51134277])

# Test cases

# loop_event(df_raw[df_raw.shopid==42818])
# loop_event(df_raw[df_raw.shopid==8996761])
loop_event(df_raw[df_raw.shopid==51134277])
# loop_event(test_df)

In [ ]:
final = df_raw.groupby('shopid').apply(loop_event)
final = final.reset_index()
final.columns = ['shopid', 'userid']

In [ ]:
final.head()
final.to_csv('result.csv', index=False)

In [ ]:
final.info()